# Proyectos bitbucket

## 📦Obtener los datos

### 1. Repositorios

Consultar los repositorios

In [482]:
%pip install python-dotenv

import requests
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import display


# Configura tus credenciales
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")


url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

response = requests.get(url_repos, auth=(username, app_password))

if response.status_code == 200:
    repositorios = response.json()
    print(f"Repositorios en {workspace}:")
    repos_data = []
    for repo in repositorios.get("values", []):
        repos_data.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
            }
        )
    repos_df = pd.DataFrame(repos_data)
    repos_df = repos_df.sort_values(
        by="Nombre"
    )  # Ordenar por nombre del repositorio en ascendente
    # print(repos_df)
else:
    print("Error al obtener repositorios:", response.status_code, response.text)

# Almacenar los repositorios en una variable para uso posterior
repositorios = [
    repo["Nombre"]
    for repo in repos_data
    if repo["Nombre"] not in ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
]



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Repositorios en itmz:


Listado de repositorios

In [483]:
repos_df = pd.DataFrame(repositorios, columns=["Repositorio"])
display(repos_df)

,Repositorio
0,api_it_management
1,b2b
2,ApiEmpleados
3,fip
4,api_data_ERP


### 2. Funciones

In [484]:
# Función para obtener todos los elementos paginados
def get_all_items(url, params=None):
    items = []
    response = requests.get(url, params=params, auth=(username, app_password))
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return items
    data = response.json()
    items.extend(data.get("values", []))
    return items


### 3. Pull requests

Consultar todos los estados de PR's.

Filtrar 50 registros por páginas, 3 páginas

In [485]:
cantidad_paginas = 2
registros_por_pagina = 50  # Creo que no soporta más de 50 registros por página

# Obtener pull requests
pull_requests = []

# for repo_slug in repositorios:
for repo_slug in ["fip", "b2b"]:
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"
    for page in range(1, cantidad_paginas + 1):
        pull_requests.extend(
            get_all_items(
                pr_url,
                params={"page": page, "pagelen": registros_por_pagina, "state": "ALL"},
            )
        )
        print(f"Obteniendo pull requests, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de pull requests: {len(pull_requests)}\n")


# Obtener commits
commits = []

for repo_slug in ["fip", "b2b"]:
    commits_url = (
        f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/commits"
    )
    for page in range(1, cantidad_paginas + 1):
        commits.extend(
            get_all_items(
                commits_url,
                params={"page": page, "pagelen": registros_por_pagina},
            )
        )
        print(f"Obteniendo commits, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de commits: {len(commits)}")

Obteniendo pull requests, repositorio: 'fip' - página: 1
Obteniendo pull requests, repositorio: 'fip' - página: 2
Obteniendo pull requests, repositorio: 'b2b' - página: 1
Obteniendo pull requests, repositorio: 'b2b' - página: 2
✅Total de pull requests: 200

Obteniendo commits, repositorio: 'fip' - página: 1
Obteniendo commits, repositorio: 'fip' - página: 2
Obteniendo commits, repositorio: 'b2b' - página: 1
Obteniendo commits, repositorio: 'b2b' - página: 2
✅Total de commits: 200


Filtrar registros de más de 60 días

In [486]:
# Filtrar los registros cuya fecha sea mayor a 60 días
cantidad_dias = 60


print(f"\nTotal pull requests: {len(pull_requests)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
pull_requests = [
    pr for pr in pull_requests if pd.to_datetime(pr["created_on"]) >= fecha_limite
]
print(f"Registros de los últimos {cantidad_dias} días: {len(pull_requests)}\n")


print(f"\nTotal commits: {len(commits)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
commits = [
    commit
    for commit in commits
    if pd.to_datetime(commit["date"]) >= fecha_limite
    and "Resolve:" in commit.get("message", "")
]
print(
    f"Registros de los últimos {cantidad_dias} días y con 'Resolve:' en el mensaje: {len(commits)}\n"
)


Total pull requests: 200
Registros de los últimos 60 días: 103


Total commits: 200
Registros de los últimos 60 días y con 'Resolve:' en el mensaje: 112



Limpiar **pull requests**:

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [487]:
for pr in pull_requests:
    if "author" in pr and "nickname" in pr["author"]:
        pr["author"] = pr["author"]["nickname"]
    if "source" in pr and "branch" in pr["source"] and "name" in pr["source"]["branch"]:
        pr["branch"] = pr["source"]["branch"]["name"]
        pr["type_branch"] = (
            pr["branch"].split("/")[0] if "/" in pr["branch"] else "unknown"
        )
    if (
        "source" in pr
        and "repository" in pr["source"]
        and "name" in pr["source"]["repository"]
    ):
        pr["repository"] = pr["source"]["repository"]["name"]
    if pr.get("merge_commit") and "hash" in pr["merge_commit"]:
        pr["merge_commit"] = pr["merge_commit"]["hash"]

    # Calculate days_open based on the PR state
    if "state" in pr and "created_on" in pr:
        created_on = pd.to_datetime(pr["created_on"])
        if pr["state"] == "OPEN":
            # For open PRs: days between created_on and now
            days_open = (
                pd.Timestamp.now(tz="America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        elif pr["state"] == "MERGED" and "updated_on" in pr:
            # For merged PRs: days between created_on and updated_on
            updated_on = pd.to_datetime(pr["updated_on"])
            days_open = (
                updated_on.tz_convert("America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        else:
            days_open = 0
        pr["days_open"] = days_open


df_pr = pd.DataFrame(pull_requests)

# Sort by created_on in descending order to show most recent records first
df_pr = df_pr.sort_values(by="created_on", ascending=False)

# Eliminar columnas innecesarias
df_pr = df_pr.drop(
    columns=[
        "type",
        "title",
        "description",
        "reason",
        "destination",
        "summary",
        "closed_by",
        "links",
        "source",
    ]
)

Limpiar **commits**

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [488]:
for cm in commits:
    # Extract author name from nested dictionary
    if "author" in cm and "user" in cm["author"] and "nickname" in cm["author"]["user"]:
        cm["author"] = cm["author"]["user"]["nickname"]
    if "repository" in cm and "name" in cm["repository"]:
        cm["repository"] = cm["repository"]["name"]


df_commits = pd.DataFrame(commits)

# Sort by date in descending order to show most recent records first
df_commits = df_commits.sort_values(by="date", ascending=False)

Imprimir **pull requests**

In [489]:
# print(type(df_pr))
print(f"Cantidad de registros: {len(df_pr)}")
display(df_pr.head(3))

Cantidad de registros: 103


,comment_count,task_count,id,state,merge_commit,close_source_branch,author,created_on,updated_on,branch,type_branch,repository,days_open
44,0,0,631,OPEN,None,True,Heriberto Sanchez,2025-03-10T15:50:06.028787+00:00,2025-03-10T15:50:06.804586+00:00,feature/BTB-731,feature,b2b,0
42,0,0,630,OPEN,None,True,Heriberto Sanchez,2025-03-10T14:02:19.431334+00:00,2025-03-10T16:48:01.400756+00:00,feature/BTB-723,feature,b2b,0
45,0,0,629,OPEN,None,True,Camilo Medina,2025-03-08T13:41:26.072376+00:00,2025-03-10T14:58:21.632109+00:00,feature/BTB-727,feature,b2b,2


Imprimir **commits**

In [ ]:
# print(type(df_commits))
print(f"Cantidad de registros: {len(df_commits)}")
display(df_commits.head(3))

<class 'pandas.core.frame.DataFrame'>
Cantidad de registros: 112


,type,hash,date,author,message,summary,links,parents,rendered,repository
46,commit,0d7751efaca326e0f88cdffdece50ad5b77e5125,2025-03-10T16:58:23+00:00,Camilo Medina,🧪Feat: Agrega pedidos en estado ERROR AL APROB...,"{'type': 'rendered', 'raw': '🧪Feat: Agrega ped...",{'self': {'href': 'https://api.bitbucket.org/2...,[{'hash': 'b03cad96b42f6fb7e55d312d2f43d8d92c8...,"{'message': {'type': 'rendered', 'raw': '🧪Feat...",b2b
47,commit,1a6371d0597f6ff743a22190d4fb405691ef3e6a,2025-03-10T15:24:08+00:00,Heriberto Sanchez,🧪Feat (CFG/ProductoDesabastecido): Ajusta depe...,"{'type': 'rendered', 'raw': '🧪Feat (CFG/Produc...",{'self': {'href': 'https://api.bitbucket.org/2...,[{'hash': 'b03cad96b42f6fb7e55d312d2f43d8d92c8...,"{'message': {'type': 'rendered', 'raw': '🧪Feat...",b2b
48,commit,e9e08a45e79d1c9a9a9c482ea60e03fb80e03608,2025-03-10T13:57:26+00:00,Heriberto Sanchez,🧪Feat: Implementa botón regresar en el compone...,"{'type': 'rendered', 'raw': '🧪Feat: Implementa...",{'self': {'href': 'https://api.bitbucket.org/2...,[{'hash': 'b03cad96b42f6fb7e55d312d2f43d8d92c8...,"{'message': {'type': 'rendered', 'raw': '🧪Feat...",b2b


### dasd